# Azure Cosmos DB の API for MongoDB と Synapse Link をはじめる

このサンプルでは、次のタスクを実行します:

1. 従来の MongoDB クライアントを使用してデータセットを挿入します。  
1. 挿入したトランザクションデータから、分析ストアに対して集計クエリを実行します。
1. MongoSpark コネクタを使用し、別のデータセットを挿入します。
1. 両方のデータセットを統合して、集計クエリを再度実行します。

## 前提条件
1. Azure Cosmos DB で MongoDB API アカウントを作成しましたか？そうでない場合は、[Azure Cosmos DB の API for MongoDB のアカウントを作成する](https://docs.microsoft.com/azure/cosmos-db/create-cosmosdb-resources-portal#create-an-azure-cosmos-db-account)に移動します。API オプションとして MongoDB を使用してアカウントを作成してください。
1. Cosmos DBアカウントで、Synapse Link を有効にしましたか？そうでない場合は、[Azure Cosmos DB アカウントの Synapse Link を有効にする](https://docs.microsoft.com/azure/cosmos-db/configure-synapse-link#enable-synapse-link)に移動します。
1. Synapse ワークスペースを作成しましたか？そうでない場合は、[Synapse ワークスペースの作成](https://docs.microsoft.com/azure/synapse-analytics/quickstart-create-workspace)に移動します。

## Synapse Link を使用して Cosmos DB コレクションを作成する 
1. `test` という名前のデータベースを作成します。
1. `pk` というパーティションキーを使用して、 `htap` という名前のコレクションを作成します。
    - コレクションを作成するときは、必ず `Analytical store` (分析ストア)オプションを `On` に設定してください。

## コレクションを Synapse に接続する
1. Synapse Analytics ワークスペースに移動します。
1. MongoDB API アカウントの `Linked Data` 接続を作成します。
    1. `Data` ブレードの下で、+ (プラス) 記号を選択します。
    1. `Connect to external data` オプションを選択します。
    1. 次に `Azure Cosmos DB (MongoDB API)` オプションを選択します。
    1. ドロップダウンを使用するか、接続文字列を入力して、特定の Azure Cosmos DB アカウントに関するすべての情報を入力します。`Linked Data` 接続に割り当てた名前をメモしておいてください。
    - または、アカウントの概要にある接続パラメータ＝を使用することもできます。
1. `Data` ブレードの `Linked` タブでデータベースアカウントを探し、接続をテストします。
    - すべてのアカウントとコレクションを含むリストがあるはずです。
    - `Analytical Store` が有効になっているコレクションには、特有のアイコンが表示されます。

### 環境を整えましょう

この環境では、実行する Python ライブラリをインストールして使用できます。このサンプルでは、次のライブラリを Spark プールに追加する必要があります:

```
pymongo==2.8.1
aenum==2.1.2
backports-abc==0.5
bson==0.5.10
```

ライブラリを Spark プールにインポートする方法については、[こちらの記事](https://docs.microsoft.com/ja-jp/azure/synapse-analytics/spark/apache-spark-azure-portal-add-libraries)をご覧ください。このための新しい Spark プールを作成することを推奨します。

次のコマンドを実行し、すべてのライブラリが正しくインストールされたことを確認します:

In [1]:
import pip #needed to use the pip functions

for i in pip.get_installed_distributions(local_only=True):
    print(i)

# The output might be long... you can collapse it by clicking on the 'Collapse output' option on the upper left corner of the output cell.

StatementMeta(MongoSpark, 11, 3, Finished, Available)

zipp 0.6.0
zict 1.0.0
xlwt 1.2.0
XlsxWriter 0.9.6
xlrd 1.0.0
wrapt 1.11.2
widgetsnbextension 2.0.0
wheel 0.30.0
Werkzeug 0.16.0
websocket-client 0.56.0
wcwidth 0.1.7
vega-datasets 0.7.0
urllib3 1.25.6
unicodecsv 0.14.1
typing-extensions 3.7.4
traitlets 4.3.2
tqdm 4.48.2
tornado 6.0.3
torch 1.3.0
toolz 0.10.0
testpath 0.3
terminado 0.6
termcolor 1.1.0
tensorflow 1.14.0
tensorflow-estimator 1.14.0
tensorboard 1.14.0
tblib 1.4.0
tables 3.3.0
sympy 1.0
statsmodels 0.10.1
SQLAlchemy 1.1.9
spyder 3.1.4
sortedcontainers 2.1.0
sortedcollections 0.5.3
snowballstemmer 1.2.1
smart-open 1.8.4
sklearn-pandas 1.7.0
skl2onnx 1.4.9
six 1.12.0
singledispatch 3.4.0.3
simplegeneric 0.8.1
shap 0.34.0
setuptools 41.4.0
SecretStorage 3.1.1
seaborn 0.9.0
scipy 1.1.0
scikit-learn 0.20.3
scikit-image 0.15.0
s3transfer 0.2.1
ruamel.yaml 0.15.89
rope-py3k 0.9.4.post1
retrying 1.3.3
Resource 0.2.1
requests 2.22.0
requests-oauthlib 1.2.0
QtPy 1.2.1
qtconsole 4.3.0
QtAwesome 0.4.4
pyzmq 16.0.2
PyYAML 5.1.2
PyWavele

### こちらにデータベースアカウント固有の秘匿情報を記述してください！

誰にも教えないようにしてください。


In [3]:
DATABASE_ACCOUNT_NAME = '<ここにテータベースアカウント名を入力します>'
DATABASE_ACCOUNT_READWRITE_KEY = '<ここにプライマリまたはセカンダリパスワードを入力します>'

StatementMeta(MongoSpark, 9, 4, Finished, Available)

## MongoDB クライアントを初期化しましょう

アカウントの概要から次のパラメータのみが必要になります: 
- 接続文字列。
- プライマリまたはセカンダリの読み取り/書き込みキー。

データベースには `test` 、コレクションには `htap` という名前を付けたことを思い出してください。

以下のコードスニペットは、`MongoClient` オブジェクトを初期化する方法を示しています。

In [4]:
from pymongo import MongoClient
from bson import ObjectId # ObjectId が機能するため

client = MongoClient("mongodb://{account}.mongo.cosmos.azure.com:10255/?ssl=true&replicaSet=globaldb".format(account = DATABASE_ACCOUNT_NAME)) # 独自のデータベースアカウントのエンドポイント。
db = client.test    # データベースを選択
db.authenticate(name=DATABASE_ACCOUNT_NAME,password=DATABASE_ACCOUNT_READWRITE_KEY) # データベースアカウント名と任意の読み取り/書き込みキーを使用します。

StatementMeta(MongoSpark, 9, 5, Finished, Available)

ModuleNotFoundError: No module named 'pymongo'

## MongoClient ドライバーを使用したデータの挿入

次のサンプルは、ランダムデータに基づいて 500 アイテムを生成します。各アイテムには次のフィールドが含まれます:
- Item, string
- Price, float
- Rating, integer
- Timestamp, [epoch integer](http://unixtimestamp.50x.eu/about.php)

このセルは、上記のセルに依存して、Cosmos DB MongoDB API アカウントへの接続のインスタンスを作成します。

このデータは、データベースの MongoDB ストアに挿入されます。これは、アプリケーションが生成するトランザクションデータをエミュレートします。

In [ ]:
from random import randint
import time

orders = db["htap"]

items = ['Pizza','Sandwich','Soup', 'Salad', 'Tacos']
prices = [2.99, 3.49, 5.49, 12.99, 54.49]

for x in range(1, 501):
    order = {
        'item' : items[randint(0, (len(items)-1))],
        'price' : prices[randint(0, (len(prices)-1))],
        'rating' : randint(1, 5),
        'timestamp' : time.time()
    }
    
    result=orders.insert(order)

print('500 個の注文の作成が終了しました')


## 分析ストアからデータを読み取る

トランザクションデータを挿入したので、分析ストアからデータを読み取ってみましょう。

データは自動的に列形式に変換されるため、大規模な集計クエリをすばやく簡単に実行できます。


In [1]:
# 分析ストアのデータをデータフレームにロードする
# シークレットを使用してセルを実行し、DATABASE_ACCOUNT_NAME および DATABASE_ACCOUNT_READWRITE_KEY 変数を取得します。
df = spark.read.format("cosmos.olap")\
    .option("spark.cosmos.accountEndpoint", "https://{account}.documents.azure.com:443/".format(account = DATABASE_ACCOUNT_NAME))\
    .option("spark.cosmos.accountKey", DATABASE_ACCOUNT_READWRITE_KEY)\
    .option("spark.cosmos.database", "test")\
    .option("spark.cosmos.container", "htap")\
    .load()

# ピザの注文からのすべての収益を調べてみましょう
df.groupBy(df.item.string).sum().show()

# df[df.item.string == 'Pizza'].show(10) 
# df.select(df['item'] == Struct).show(10) 
# df.select("timestamp.float64").show(10)
#df.select("timestamp.string", when(df.timestamp.string != null)).show(10)

StatementMeta(, , , SessionStarting, )

## 分析ストアの MongoDB スキーマに関する簡単なメモ

Mongoアカウントでは、 **完全な忠実度スキーマ**を使用します。これは、データ型で拡張されたプロパティ名の表現であり、値の正確な表現を提供し、あいまいさを回避します。
これが、上記のフィールドを呼び出すときに、それらのデータ型をサフィックスとして使用した理由です。 以下の例のように:

```
df.filter((df.item.string == "Pizza")).show(10)
```
プロパティの名前の後に文字列型を指定したことに注意してください。 以下は、分析ストア内のすべての潜在的なプロパティとそれらのサフィックス表現のマップです:

| Original Data Type     &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;| Suffix    &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;| Example &nbsp;&nbsp;&nbsp;&nbsp; | 
|---------------|----------------|--------|
| Double        | ".float64"     |  `24.99`   |
| Array         | ".array"       |  `["a", "b"]`   |
| Binary        | ".binary"      |  `0`   |
| Boolean       | ".bool"        |  `True`   |
| Int32         | ".int32"       |  `123`   |
| Int64         | ".int64"       |  `255486129307`   |
| Null          | ".null"        |  `null`   |
| String        | ".string"      |  `"ABC"`   |
| Timestamp     | ".timestamp"   |  `Timestamp(0, 0)`   |
| DateTime      | ".date"        |  `ISODate("2020-08-21T07:43:07.375Z")`   |
| ObjectId      | ".objectId"    |  `ObjectId("5f3f7b59330ec25c132623a2")`   |
| Document      | ".object"      |  `{"a": "a"}`   |

これらの型は、トランザクションストアに挿入されたデータから推測されます。 次のコマンドを実行してスキーマを確認できます:

```
df.schema
```
```

## さらに注文を入れましょう！

今回は少し異なるデータを使用します。各アイテムには次のフィールドが含まれます:
- Item, string
- Price, float
- Rating, integer
- Timestamp, [ISO String format](https://en.wikipedia.org/wiki/ISO_8601)

`Timestamp` フィールドが文字列形式になっていることに注意してください。これは、データタイプに基づいてさまざまなデータフィールドを読み取る方法を理解するのに役立ちます。

In [ ]:
from random import randint
from time import strftime

orders = db["htap"]

items = ['Pizza','Sandwich','Soup', 'Salad', 'Tacos']
prices = [2.99, 3.49, 5.49, 12.99, 54.49]

for x in range(1, 501):
    order = {
        'item' : items[randint(0, (len(items)-1))],
        'price' : prices[randint(0, (len(prices)-1))],
        'rating' : randint(1, 5),
        'timestamp' : strftime("%Y-%m-%d %H:%M:%S")
    }
    
    result=orders.insert(order)

print('500 個の注文の作成が終了しました')

## 注文データをもう一度読み取りましょう！

今回は、`timestamp.string` パラメーターを指定して、ISO 文字列の日付を個別に読み取ります。


In [26]:
# 分析ストアのデータをデータフレームにロードする
# シークレットを使用してセルを実行し、DATABASE_ACCOUNT_NAME および DATABASE_ACCOUNT_READWRITE_KEY 変数を取得します。
df = spark.read.format("cosmos.olap")\
    .option("spark.cosmos.accountEndpoint", "https://{account}.documents.azure.com:443/".format(account = DATABASE_ACCOUNT_NAME))\
    .option("spark.cosmos.accountKey", DATABASE_ACCOUNT_READWRITE_KEY)\
    .option("spark.cosmos.database", "test")\
    .option("spark.cosmos.container", "htap")\
    .load()

# ピザの注文からのすべての収益を調べてみましょう
df.filter( (df.timestamp.string != "")).show(10)

StatementMeta(MongoSpark, 9, 27, Finished, Available)

+--------------------+----------+--------------------+--------------------+--------------+----------+-------+------+--------------------+---+--------------+
|                _rid|       _ts|                  id|               _etag|           _id|      item|  price|rating|           timestamp| pk| _partitionKey|
+--------------------+----------+--------------------+--------------------+--------------+----------+-------+------+--------------------+---+--------------+
|c8dVAMNlPsb1AQAAA...|1597792391|NWYzYzYwODdmYjVkM...|"00003909-0000-08...|[_<`��]RUj�]|   [Tacos]|[12.99]|   [1]|[, 2020-08-18 23:...|[3]|[c8dVAMNlPsY=]|
|c8dVAMNlPsb2AQAAA...|1597792391|NWYzYzYwODdmYjVkM...|"00003a09-0000-08...|[_<`��]RUj�]|[Sandwich]|[54.49]|   [5]|[, 2020-08-18 23:...|[1]|[c8dVAMNlPsY=]|
|c8dVAMNlPsb3AQAAA...|1597792391|NWYzYzYwODdmYjVkM...|"00003b09-0000-08...|[_<`��]RUj�]|    [Soup]| [5.49]|   [4]|[, 2020-08-18 23:...|[4]|[c8dVAMNlPsY=]|
|c8dVAMNlPsb4AQAAA...|1597792391|NWYzYzYwODdmYjVkM...|"000